In [ ]:
import numpy as np
import plotly.graph_objects as go
from PIL import Image

# Konstanter
G = 6.67430e-11  # Gravitationskonstanten, m^3 kg^-1 s^-1
M_moon = 7.342e22  # Månens masse, kg
R_moon = 1.737e6  # Månens radius, m
mu = G * M_moon  # Standard gravitasjonsparameter, m^3 s^-2

# Orbitalparametere
a = 2000000  # Semi-major axis, m
e = 0.1  # Eccentricity
i = np.radians(30)  # Inclination, radianer
Ω = np.radians(40)  # Right ascension of the ascending node, radianer
ω = np.radians(60)  # Argument of periapsis, radianer
ν = np.linspace(0, 2 * np.pi, 1000)  # True anomaly, radianer

# Beregn posisjoner i baneplanet
r = a * (1 - e**2) / (1 + e * np.cos(ν))
x_orbit = r * np.cos(ν)
y_orbit = r * np.sin(ν)
z_orbit = np.zeros_like(ν)

# Rotasjonsmatriser
R_Ω = np.array([
    [np.cos(Ω), -np.sin(Ω), 0],
    [np.sin(Ω), np.cos(Ω), 0],
    [0, 0, 1]
])

R_i = np.array([
    [1, 0, 0],
    [0, np.cos(i), -np.sin(i)],
    [0, np.sin(i), np.cos(i)]
])

R_ω = np.array([
    [np.cos(ω), -np.sin(ω), 0],
    [np.sin(ω), np.cos(ω), 0],
    [0, 0, 1]
])

# Kombinert rotasjonsmatrise
R = R_Ω @ R_i @ R_ω

# Roter baneplanet til 3D-rommet
orbit_3d = R @ np.vstack((x_orbit, y_orbit, z_orbit))

# Generer sfæren for månen
phi, theta = np.mgrid[0:np.pi:100j, 0:2*np.pi:100j]
x_moon = R_moon * np.sin(phi) * np.cos(theta)
y_moon = R_moon * np.sin(phi) * np.sin(theta)
z_moon = R_moon * np.cos(phi)

# Last inn teksturbildet av månen
img = Image.open('moonmap4k.jpg')
texture = np.asarray(img).T  # Transponer bildet for riktig mapping

# Definer fargeskalaen
colorscale = [[0.0, 'rgb(30, 30, 30)'],
              [0.5, 'rgb(150, 150, 150)'],
              [1.0, 'rgb(255, 255, 255)']]

# Plot banen i 3D med plotly
fig = go.Figure()

# Legg til satellittbanen
fig.add_trace(go.Scatter3d(
    x=orbit_3d[0], y=orbit_3d[1], z=orbit_3d[2],
    mode='lines',
    name='Satellittbane'
))

# Legg til månen som en sfære med tekstur
fig.add_trace(go.Surface(
    x=x_moon, y=y_moon, z=z_moon,
    opacity=1,
    showscale=False,
    name='Månen'
))

# Oppsett av aksene
fig.update_layout(
        title=dict(
            text='Satellittbane rundt månen',
            font=dict(size=24, color='white'),
            x=0.5, y=0.95, xanchor='center', yanchor='top'
        ),
        width=800,
        height=600,
        scene=dict(
            xaxis=dict(backgroundcolor='black', color='white', gridcolor='gray', title='x (m)'),
            yaxis=dict(backgroundcolor='black', color='white', gridcolor='gray', title='y (m)'),
            zaxis=dict(backgroundcolor='black', color='white', gridcolor='gray', title='z (m)'),
            bgcolor='black'
        ),
        legend=dict(
            x=0, y=1, xanchor='left', yanchor='top', font=dict(size=10, color="white")
        ),
        margin=dict(r=1, l=1, b=1, t=1),
        paper_bgcolor='black',
        plot_bgcolor='black'
    )

fig.show()